# Data preparation and transformation exercise

### Part I - Load and Inspect

The objective of this exercise is to practice various steps of data preprocessing and feature engineering.

The scenario is the preparation of data for a ML multilinear regressions.

The dataset used is the "Climate Weather Surface of Brazil - Hourly", wich is available at <a href="https://www.kaggle.com/PROPPG-PPG/hourly-weather-surface-brazil-southeast-region?select=make_dataset.py">Kaggle</a>.

It contains hourly climate data taken from 122 weather stations in Brasil between 2000 and 2021.

**Steps:**
1. <a href="#Load-data">Load data</a>
2. <a href="#Inspect-data">Inspect data</a>
3. Format features
4. Clean messy data
5. Remove duplicate values
6. Treat missing values
7. Imputation
8. Remove strongly correlated features
9. Remove outliers
10. Aggregate features
11. Encode categorical features
12. Feature scaling
13. Dimensionality reduction and feature decomposition
14. Sample and balance

## About the dataset

| Original position | Column Name | Description | Position on load |
|--|--|--|--|
| 0 | date | date (YYYY-MM-DD) | not loaded |
| 1 | hour | hour (HH:00) | not loaded |
| NA | full_time | date + time | 0 |
| 2 | total precipitation (mm) | Amount of precipitation in millimetres (last hour) | 1 |
| 3 | atmospheric pressure at station height (mb) | Atmospheric pressure at station level (mb) | 2 |
| 4 | atmospheric pressure max. in the previous hour (mb) | Maximum air pressure for the last hour in hPa to tenths | 3 |
| 5 | atmospheric pressure min. in the previous hour (mb) | Minimum air pressure for the last hour in hPa to tenths | 4 |
| 6 | radiation (kj/m2) | Solar radiation KJ/m2 | 5 |
| 7 | air temperature - dry bulb (¬∞c) | Air temperature (instant) in celsius degrees | 6 |
| 8 | dew point temperature (¬∞c) | Dew point temperature (instant) in celsius degrees | 7 |
| 9 | max. temperature in the previous hour (¬∞c) | Maximum temperature for the last hour in celsius degrees | 8 |
| 10 | min. temperature in the previous hour (¬∞c) | Minimum temperature for the last hour in celsius degrees | 9 |
| 11 | dew temperature max. in the previous hour (¬∞c) | Maximum dew point temperature for the last hour in celsius degrees | 10 |
| 12 | dew temperature min. in the previous hour (¬∞c) | Minimum dew point temperature for the last hour in celsius degrees | 11 |
| 13 | relative humidity max. in the previous hour (%) | Maximum relative humid temperature for the last hour in % | 12 |
| 14 | relative humidity min. in the previous hour (%) | Minimum relative humid temperature for the last hour in % | 13 |
| 15 | air relative humidity (%) | Relative humid in % (instant) | 14 |
| 16 | wind direction (¬∞ (gr)) | Wind direction in radius degrees (0-360) | 15 |
| 17 | wind rajada maxima (m/s) | Wind gust in metres per second | 16 |
| 18 | wind speed (m/s) | Wind speed in metres per second | 17 |
| 19 | region | Brazilian geopolitical regions | 18 |
| 20 | state | State (Province) | 19 |
| 21 | station | Name station (usually city location or nickname) | 20 |
| 22 | station_code | Station number (INMET number) for the location | 21 |
| 23 | latitude | Latitude | 22 |
| 24 | longitude | Longitude | 23 |

## Load data

In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt

In [2]:
# automatic feature type detection is resource intensive, so we set the types manually.
dataset = pd.read_csv('./dataset/archive/central_west.csv',
                      index_col="index",
                      parse_dates={'full_time' : [1, 2]},
                      dtype={'Data': object,
                             'Hora': object,
                             'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)': np.float64,
                             'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)': np.float64,
                             'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)': np.float64,
                             'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)': np.float64,
                             'RADIACAO GLOBAL (Kj/m²)': np.int64,
                             'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)': np.float64,
                             'TEMPERATURA DO PONTO DE ORVALHO (°C)': np.float64,
                             'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)': np.float64,
                             'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)': np.float64,
                             'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)': np.float64,
                             'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)': np.float64,
                             'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)': np.int64,
                             'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)': np.int64,
                             'UMIDADE RELATIVA DO AR, HORARIA (%)': np.int64,
                             'VENTO, DIREÇÃO HORARIA (gr) (° (gr))': np.int64,
                             'VENTO, RAJADA MAXIMA (m/s)': np.float64,
                             'VENTO, VELOCIDADE HORARIA (m/s)': np.float64,
                             'region': object,
                             'state': object,
                             'station': object,
                             'station_code': object,
                             'latitude': np.float64,
                             'longitude': np.float64,
                             'height': np.float64},
                      names=["index",
                             "date", 
                             "time", 
                             "precipitation", 
                             "pressure", 
                             "pressure_max", 
                             "pressure_min", 
                             "solar_radiation", 
                             "air_temperature", 
                             "dp_temperature", 
                             "air_temp_max", 
                             "air_temp_min", 
                             "dp_temp_max", 
                             "dp_temp_min", 
                             "rel_hum_max", 
                             "rel_hum_min", 
                             "Rel_humidity", 
                             "wind_direction", 
                             "wind_gust", 
                             "wind_speed",
                             "region", 
                             "state", 
                             "station_name", 
                             "station_id", 
                             "Latitude", 
                             "Longitude", 
                             "Elevation"],
                     header = 0)

In [3]:
pickle.dump(dataset, open("raw_dataset.pkl", "wb"))

In [4]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
dataset = pickle.load(open("raw_dataset.pkl", "rb"))

## Inspect data

In [5]:
dataset

,full_time,precipitation,pressure,pressure_max,pressure_min,solar_radiation,air_temperature,dp_temperature,air_temp_max,air_temp_min,...,wind_direction,wind_gust,wind_speed,region,state,station_name,station_id,Latitude,Longitude,Elevation
index,,,,,,,,,,,,,,,,,,,,,
138998,2017-12-20 14:00:00,0.0,899.6,900.0,899.6,3391,26.5,17.7,26.5,24.4,...,39,9.6,3.9,CO,DF,PARANOA (COOPA-DF),A047,-16.011111,-47.5575,1043.0
138999,2017-12-20 15:00:00,0.0,899.2,899.6,899.2,3306,26.6,16.7,26.7,25.4,...,55,8.3,3.4,CO,DF,PARANOA (COOPA-DF),A047,-16.011111,-47.5575,1043.0
139000,2017-12-20 16:00:00,0.0,898.6,899.2,898.6,3167,27.3,15.8,27.6,25.9,...,62,8.3,3.6,CO,DF,PARANOA (COOPA-DF),A047,-16.011111,-47.5575,1043.0
139001,2017-12-20 17:00:00,0.0,897.7,898.6,897.7,3279,27.5,12.9,28.5,26.6,...,43,6.7,3.1,CO,DF,PARANOA (COOPA-DF),A047,-16.011111,-47.5575,1043.0
139002,2017-12-20 18:00:00,0.0,897.0,897.7,897.0,2753,27.5,13.7,28.9,27.4,...,98,6.4,3.5,CO,DF,PARANOA (COOPA-DF),A047,-16.011111,-47.5575,1043.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138993,2017-12-20 09:00:00,0.0,898.8,898.8,898.5,29,18.4,15.8,18.4,17.6,...,72,3.5,1.7,CO,DF,PARANOA (COOPA-DF),A047,-16.011111,-47.5575,1043.0
138994,2017-12-20 10:00:00,0.0,899.4,899.4,898.8,557,20.3,16.9,20.3,18.4,...,69,3.5,1.8,CO,DF,PARANOA (COOPA-DF),A047,-16.011111,-47.5575,1043.0
138995,2017-12-20 11:00:00,0.0,899.7,899.7,899.4,1441,21.8,16.6,21.9,20.3,...,83,7.6,4.4,CO,DF,PARANOA (COOPA-DF),A047,-16.011111,-47.5575,1043.0


### Determine/confirm initial feature types

In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11427120 entries, 138998 to 138997
Data columns (total 25 columns):
 #   Column           Dtype         
---  ------           -----         
 0   full_time        datetime64[ns]
 1   precipitation    float64       
 2   pressure         float64       
 3   pressure_max     float64       
 4   pressure_min     float64       
 5   solar_radiation  int64         
 6   air_temperature  float64       
 7   dp_temperature   float64       
 8   air_temp_max     float64       
 9   air_temp_min     float64       
 10  dp_temp_max      float64       
 11  dp_temp_min      float64       
 12  rel_hum_max      int64         
 13  rel_hum_min      int64         
 14  Rel_humidity     int64         
 15  wind_direction   int64         
 16  wind_gust        float64       
 17  wind_speed       float64       
 18  region           object        
 19  state            object        
 20  station_name     object        
 21  station_id       object   

The types are as requested, so no strings existed in float or integer features.

In [7]:
dataset.nunique(axis=0)

full_time          183936
precipitation         370
pressure             1605
pressure_max         1701
pressure_min         1692
solar_radiation     29575
air_temperature       515
dp_temperature        507
air_temp_max          516
air_temp_min          511
dp_temp_max           531
dp_temp_min           491
rel_hum_max            97
rel_hum_min            97
Rel_humidity           95
wind_direction        361
wind_gust             289
wind_speed            180
region                  1
state                   4
station_name          116
station_id            115
Latitude              230
Longitude             233
Elevation             198
dtype: int64

The features 'region', 'state', 'station_name' / 'station_id' are categorical.

We have data from 116 different weather stations, but one is missing its ID.

In [8]:
print ("We have data from {} , up to {}.".format(str(dataset.full_time.min()), str(dataset.full_time.max())))

We have data from 2000-05-07 00:00:00 , up to 2021-04-30 23:00:00.


In [9]:
first_day = dataset[dataset.full_time == '2000-05-07 00:00:00'].full_time.value_counts()
last_day = dataset[dataset.full_time == '2021-04-30 23:00:00'].full_time.value_counts()
print ("We have {} data points on the first day of the dataset and {}  in the last.".format(str(first_day[0]),
                                                                                            str(last_day[0])))

We have 1 data points on the first day of the dataset and 113  in the last.


Let's see the number of stations per year.

In [10]:
# we'll change the type of 'date' feature later. For now we'll just extract the year
dataset['year'] = dataset.full_time.dt.year

In [11]:
dataset.groupby('year')['station_name'].nunique()

year
2000      1
2001      6
2002     11
2003     14
2004     14
2005     14
2006     36
2007     54
2008     76
2009     76
2010     78
2011     81
2012     84
2013     86
2014     87
2015     87
2016     87
2017     93
2018    112
2019    115
2020    114
2021    113
Name: station_name, dtype: int64

This dataset started with a signle station and had 113 stations by 2021.

In [12]:
dataset.index.is_unique

False

There are duplicates in the index data loaded from the CSV file.

In [13]:
dataset[dataset.index==0].head()

,full_time,precipitation,pressure,pressure_max,pressure_min,solar_radiation,air_temperature,dp_temperature,air_temp_max,air_temp_min,...,wind_gust,wind_speed,region,state,station_name,station_id,Latitude,Longitude,Elevation,year
index,,,,,,,,,,,,,,,,,,,,,
0,2018-09-06 18:00:00,0.0,899.8,900.7,899.8,2659,31.7,3.2,32.5,30.3,...,6.2,1.6,CO,DF,AGUAS EMENDADAS,A045,-15.596389,-47.625833,1200.00,2018
0,2018-04-15 22:00:00,0.0,976.0,976.2,976.0,55,25.3,21.7,26.4,25.3,...,1.0,0.0,CO,MT,NOVA MARINGA,A928,-13.065556,-57.156111,353.00,2018
0,2018-04-05 00:00:00,-9999.0,-9999.0,-9999.0,-9999.0,-9999,-9999.0,-9999.0,-9999.0,-9999.0,...,-9999.0,-9999.0,CO,MS,SELVIRIA,S717,-20.351389,-51.430278,374.00,2018
0,2019-04-13 13:00:00,0.2,918.5,918.5,918.2,951,23.0,21.3,23.0,22.0,...,0.0,0.0,CO,MT,ALTO TAQUARI,A934,-17.841111,-53.289444,862.42,2019
0,2019-01-01 00:00:00,0.0,974.7,974.7,973.3,-9999,25.0,20.7,25.6,25.0,...,4.9,2.0,CO,GO,ARAGARCAS,A013,-15.902658,-52.245172,327.35,2019


In [14]:
# Let's reset the index
dataset.reset_index(inplace=True)
dataset.drop(columns = ["index"], inplace=True)

In [15]:
dataset

,full_time,precipitation,pressure,pressure_max,pressure_min,solar_radiation,air_temperature,dp_temperature,air_temp_max,air_temp_min,...,wind_gust,wind_speed,region,state,station_name,station_id,Latitude,Longitude,Elevation,year
0,2017-12-20 14:00:00,0.0,899.6,900.0,899.6,3391,26.5,17.7,26.5,24.4,...,9.6,3.9,CO,DF,PARANOA (COOPA-DF),A047,-16.011111,-47.5575,1043.0,2017
1,2017-12-20 15:00:00,0.0,899.2,899.6,899.2,3306,26.6,16.7,26.7,25.4,...,8.3,3.4,CO,DF,PARANOA (COOPA-DF),A047,-16.011111,-47.5575,1043.0,2017
2,2017-12-20 16:00:00,0.0,898.6,899.2,898.6,3167,27.3,15.8,27.6,25.9,...,8.3,3.6,CO,DF,PARANOA (COOPA-DF),A047,-16.011111,-47.5575,1043.0,2017
3,2017-12-20 17:00:00,0.0,897.7,898.6,897.7,3279,27.5,12.9,28.5,26.6,...,6.7,3.1,CO,DF,PARANOA (COOPA-DF),A047,-16.011111,-47.5575,1043.0,2017
4,2017-12-20 18:00:00,0.0,897.0,897.7,897.0,2753,27.5,13.7,28.9,27.4,...,6.4,3.5,CO,DF,PARANOA (COOPA-DF),A047,-16.011111,-47.5575,1043.0,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11427115,2017-12-20 09:00:00,0.0,898.8,898.8,898.5,29,18.4,15.8,18.4,17.6,...,3.5,1.7,CO,DF,PARANOA (COOPA-DF),A047,-16.011111,-47.5575,1043.0,2017
11427116,2017-12-20 10:00:00,0.0,899.4,899.4,898.8,557,20.3,16.9,20.3,18.4,...,3.5,1.8,CO,DF,PARANOA (COOPA-DF),A047,-16.011111,-47.5575,1043.0,2017
11427117,2017-12-20 11:00:00,0.0,899.7,899.7,899.4,1441,21.8,16.6,21.9,20.3,...,7.6,4.4,CO,DF,PARANOA (COOPA-DF),A047,-16.011111,-47.5575,1043.0,2017
11427118,2017-12-20 12:00:00,0.0,899.8,899.8,899.7,2334,23.2,17.3,23.3,21.7,...,7.6,3.7,CO,DF,PARANOA (COOPA-DF),A047,-16.011111,-47.5575,1043.0,2017


In [16]:
pickle.dump(dataset, open("newindex_dataset.pkl", "wb"))

### Inspect categorical features

In [18]:
# Determine which Brazilian states have values
states = dataset.state.unique()
print(states)

['DF' 'MS' 'MT' 'GO']


In [19]:
# Determine number of weather stations per state
dataset.groupby('state')['station_name'].nunique()

state
DF     5
GO    26
MS    45
MT    40
Name: station_name, dtype: int64

Let's check if there are different values for latitude, longitude and elevation for the same station.

In [20]:
dataset.groupby("station_name")['Latitude'].nunique().max()

3

In [21]:
dataset.groupby("station_name")['Longitude'].nunique().max()

3

In [22]:
dataset.groupby("station_name")['Elevation'].nunique().max()

2

Yes, we have different values for some stations.

In [23]:
# let's look at an example
duplicated_station_name = dataset.groupby("station_name")['Latitude'].nunique().index[0]
duplicated_station_name

'AGUA BOA'

In [24]:
dataset.loc[dataset.station_name == duplicated_station_name, ['Latitude', 
                                                              'Longitude', 
                                                              'Elevation']].drop_duplicates()

,Latitude,Longitude,Elevation
938104,-14.016111,-52.212222,432.0
1727002,-14.016389,-52.211667,440.0


These two values for Elevation must be due to a correction of the elevation value associated with the station (assuming the station wasn't raised or lowered by eight meters.

### Find missing values

In [25]:
dataset.isnull().sum()

full_time          0
precipitation      0
pressure           0
pressure_max       0
pressure_min       0
solar_radiation    0
air_temperature    0
dp_temperature     0
air_temp_max       0
air_temp_min       0
dp_temp_max        0
dp_temp_min        0
rel_hum_max        0
rel_hum_min        0
Rel_humidity       0
wind_direction     0
wind_gust          0
wind_speed         0
region             0
state              0
station_name       0
station_id         0
Latitude           0
Longitude          0
Elevation          0
year               0
dtype: int64

There appear to be no missing values, but we saw earlier that we had one fewer station_id than station_name.

In [26]:
# Let's see if the same station has multiple IDs
station_list = dataset.iloc[:,[20,21]].value_counts().to_frame()
station_list.reset_index(inplace=True)
station_list.drop(columns=0, inplace=True)

In [27]:
station_list

,station_name,station_id
0,BRASILIA,A001
1,MORRINHOS,A003
2,GOIANIA,A002
3,TRES LAGOAS,A704
4,PONTA PORA,A703
...,...,...
111,PORANGATU,A005
112,BRASNORTE (NOVO MUNDO),A927
113,SAO JOSE DO XINGU,A942
114,SERRA NOVA DOURADA,A943


In [28]:
station_list.sort_values(by='station_name')

,station_name,station_id
32,AGUA BOA,A908
76,AGUA CLARA,A756
73,AGUAS EMENDADAS,A045
41,ALTA FLORESTA,A924
80,ALTO ARAGUAIA,A909
...,...,...
82,SONORA,A761
9,SORRISO,A904
7,TANGARA DA SERRA,A902
3,TRES LAGOAS,A704


In [29]:
station_list.station_id.value_counts()

A927    2
A001    1
A046    1
A759    1
A760    1
       ..
A915    1
A732    1
A908    1
A731    1
A944    1
Name: station_id, Length: 115, dtype: int64

The station with 'station_id' == A927 has two different names ("BRASNORTE (NOVO MUNDO)" and "BRASNORTE (MUNDO NOVO)"). We'll standardize that later.